<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/FineTune_text_summarization_with_lora_and__4bit_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope of this notebook
1. Load LLaMA in 4-bit quantization
2. Fine-Tune LLaMa with QLoRA for text summarization

In [1]:
!pip install bitsandbytes
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 

### Library

In [15]:
import time
import numpy as np
import random


import torch


from transformers import TraningArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset, load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

import evaluate

## Config

In [16]:
def get_config():
  return {
      'dataset': 'knkarthick/dialogsum',
      'model_id': 'NousResearch/Llama-2-7b-hf'
      'output_dir': '',
  }

### load dataset and convert it into instruction finetuning formate

In [17]:
def instructionFormate(data):
  newData = []
  for dataPoint in data:
    inputs = f"""
      INSTRUCTION: Summarize the following conversarion.
      ### INPUT:
      {dataPoint['dialogue']}

      ### Summary:
      {dataPoint['summary']}
    """
    item = {
        "dialogue": dataPoint['dialogue'],
        "summary": dataPoint['summary'],
        "inputs": inputs
    }
    newData.append(item)
  return newData

def get_data(config):
  data = {}
  raw_data = load_dataset(config['dataset'])
  train_ds = instructionFormate(raw_data['train'])
  val_ds = instructionFormate(raw_data['validation'])
  test_ds = instructionFormate(raw_data['test'])
  data['train'] = train_ds
  data['train'] = val_ds
  data['train'] = test_ds
  return data

# config = get_config()
# train_ds, val_ds, test_ds = get_data(config)

### Load model into 4-bit Quantize form

In [18]:
def get_model_and_tokenizer(config):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  model = AutoModelForCausalLM.from_pretrained(config['model_id'], quantization_config=bnb_config, device_map="auto")
  tokenizer = AutoTokenizer.from_pretrained(config['model_id'])
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = 'right'
  return model, tokenizer


### ZERO Shot inference with LLaMA-2 7B

In [20]:
config = get_config()
data = get_data(config)
model, tokenizer = get_model_and_tokenizer(config)

dialogue = data['test'][0]['dialogue']
summary = data['test'][0]['dialogue']
prompt = data['test'][0]['inputs']

tokenized_input = tokenizer(prompt, return_tensors='pt')
encoded_output = model.generate(tokenized_input['input_ids'], max_new_tokens=100)
output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f"Dialogue: \n{dialogue}\n")
print(dash_line)
print(f"Expected Summary: \n{summary}\n")
print(dash_line)
print(f"Generated Summary: \n{output}\n")
print(dash_line)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

### Print trainable parameters function

In [ ]:
def print_trainable_parameters(model):
  trainable_param = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_param += param.numel()

  print(
      f"Trainable Param: {} || All Param: {} || trainable percentage: {100 * trainable_params / all_param}%"
  )

### load 4bit training peft version and print model

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print(model)

### Set LoRA parameters

In [19]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # name of the layer is same as mention in the model architecture
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], # layers name for which we want to finetune additional parameters
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

Lora_Model = get_peft_model(model, lora_config)
print_trainable_parameters(Lora_Model)

### Tensor board

In [ ]:
%load_ext tensorboard
%tensorboard --logdir llama2-docsum-adapter/runs

### Training

In [19]:
training_arguments = TrainingArguments(
    pre_device_train_batch_size=config['batch_size'],
    gradient_accumulation_steps=config['gradient_step'],
    optim=config['optimName'],
    logging_step=1,
    learning_rate=config['lr'],
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=config['epochs'],
    evaluation_strategy='step',
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy='epoch',
    group_by_length=True,
    output_dir=config['output_dir'],
    report_to='tensorboard',
    save_safetensors=True,
    lr_scheduler_type='cosine',
    seed=42,
)

Lora_Model.config.use_cache = False

In [19]:
trainer = SFTTrainer(
    model = Lora_Model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=lora_config,
    dataset_text_field='prompt',
    max_seq_length=config['max_len'],
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

### Save Model

In [ ]:
# here we only save the additional trainable weigts not the frozen model weigts
trainer.model.save_pretrained(config['peft_model_path'])
tokenizer.save_pretrained(config['peft_model_path'])


### Inference

In [21]:
from transformers import TextStreamer
from peft import AutoPeftModelForCausalLM
from transformer import AutoTokenizer

In [ ]:
Lora_Model.config.use_cache = True
Lora_Model.eval()

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    config['peft_model_path'],
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(config['peft_model_path'])


### Generate output from fine tune LLaMA+lora model

In [ ]:
dialogue = data['test'][0]['dialogue']
summary = data['test'][0]['dialogue']
prompt = data['test'][0]['inputs']

tokenized_input = tokenizer(prompt, return_tensors='pt')
encoded_output = trained_model.generate(tokenized_input['input_ids'], max_new_tokens=100)
output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))

print(dash_line)
print(f"Dialogue: \n{dialogue}\n")
print(dash_line)
print(f"Expected Summary: \n{summary}\n")
print(dash_line)
print(f"Generated Summary: \n{output}\n")
print(dash_line)